In [0]:
import tensorflow as tf
tf.enable_eager_execution() 
import csv
from nltk import word_tokenize
import string
import json, gzip
from pandas.io.json import json_normalize
import pandas as pd
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
def clean(text):
    text = text.lower()
    printable = set(string.printable)
    text = "".join(list(filter(lambda x: x in printable, text))) #filter funny characters, if any.
    return text

text_max_len = 2000
text_min_len = 250
summary_max_len = 40
vocab2idx = {}

In [0]:
path_train = "/content/drive/My Drive/Colab Notebooks/summ/release/train.jsonl.gz"
#path_val = "/content/drive/My Drive/Colab Notebooks/summ/release/dev.jsonl.gz"
#path_test = "/content/drive/My Drive/Colab Notebooks/summ/release/test.jsonl.gz"
data_train = []
data_val = []
data_test = []

with gzip.open(path_train) as f:
    for ln in f:
        obj = json.loads(ln)
        data_train.append(obj)
'''
with gzip.open(path_val) as f:
    for ln in f:
        obj = json.loads(ln)
        data_val.append(obj)

with gzip.open(path_test) as f:
    for ln in f:
        obj = json.loads(ln)
        data_test.append(obj)
df_val = pd.DataFrame.from_dict(json_normalize(data_val), orient='columns')
df_test = pd.DataFrame.from_dict(json_normalize(data_test), orient='columns')
'''
df_train = pd.DataFrame.from_dict(json_normalize(data_train), orient='columns')


In [0]:
data_s_train = df_train[['text', 'summary']]
data_s_train.head()
Text = data_s_train['text']
Summary = data_s_train['summary']
print(len(Summary))
print(len(Text))

995041
995041


In [0]:
txt = []
summ = []
texts = []
summaries = []
for i in range(len(Text)):
  txt = Text[i]
  summ = Summary[i]
  if len(txt) <= text_max_len and len(txt) >= text_min_len and len(summ) <= summary_max_len:
    clean_text = clean(txt)
    clean_summary = clean(summ)
    tokenized_summary = word_tokenize(clean_summary)
    tokenized_text = word_tokenize(clean_text)

    for word in tokenized_text:
      if word not in vocab2idx:
        vocab2idx[word]=len(vocab2idx)
    for word in tokenized_summary:
      if word not in vocab2idx:
       vocab2idx[word]=len(vocab2idx)
  
    summaries.append(tokenized_summary)
    texts.append(tokenized_text)

    if i%10000==0:
      print("Processing data # {}".format(i))
    i+=1
                          
print("\n# of Data: {}".format(len(texts)))


# of Data: 15828


In [0]:
import numpy, random

index = random.randint(0, len(texts)-1)
print("SAMPLE CLEANED & TOKENIZED TEXT: \n\n{}\n\n".format(texts[index]))
print("SAMPLE CLEANED & TOKENIZED SUMMARY: \n\n{}\n".format(summaries[index]))

SAMPLE CLEANED & TOKENIZED TEXT: 

['the', 'brexit', 'is', 'official', ',', 'with', '52', 'percent', 'of', 'the', 'united', 'kingdom', 'voting', 'to', 'leave', 'the', 'european', 'union', '.', 'even', 'though', 'a', 'lot', 'of', 'people', 'are', 'not', 'happy', 'about', 'the', 'decision', ',', 'and', 'its', 'strong', 'implications', 'on', 'the', 'economy', ',', 'travel', ',', 'education', ',', 'healthcare', 'and', 'culture', ',', 'many', 'wasted', 'no', 'time', 'in', 'taking', 'lemons', 'and', 'making', 'lemonade', '.', 'here', 'are', 'some', 'of', 'the', 'best', 'reactions', ',', 'brought', 'to', 'you', 'by', 'horrified', 'people', '.', 'a', 'lot', 'of', 'them', 'involve', 'crying', 'jordan', '.', 'meanwhile', 'india', 'is', 'just', 'blown', 'away', 'that', 'you', 'can', 'get', 'britain', 'to', 'leave', 'by', 'voting', 'bobby', 'big', 'wheel', '(', '@', 'bobbybigwheel', ')', 'june', '23', ',', '2016', 'have', 'something', 'to', 'add', 'to', 'this', 'story', '?', 'share', 'it', 'in', '

In [0]:
import numpy as np

vocab = []
embd = []
special_tags = ['<UNK>','<PAD>','<EOS>']


def loadEmbeddings(filename):
    vocab2embd = {}
    
    with open(filename) as infile:     
        for line in infile:
            row = line.strip().split(' ')
            word = row[0].lower()
            if word not in vocab2embd:
                vocab2embd[word]=np.asarray(row[1:],np.float32)

    print('Embedding Loaded.')
    return vocab2embd

vocab2embd = loadEmbeddings('/content/drive/My Drive/Colab Notebooks/Data/glove/glove.6B.100d.txt')

for word in vocab2idx:
    if word in vocab2embd:
        vocab.append(word)
        embd.append(vocab2embd[word])
        
for special_tag in special_tags:
    vocab.append(special_tag)
    embd.append(np.random.rand(len(embd[0]),))
    
vocab2idx = {word:idx for idx,word in enumerate(vocab)}
embd = np.asarray(embd,np.float32)

print("Vocabulary Size: {}".format(len(vocab2idx)))

Embedding Loaded.
Vocabulary Size: 70150


In [0]:
print(vocab2idx['<EOS>'])

70149


In [0]:
vec_texts=[]
vec_summaries=[]

for text,summary in zip(texts,summaries):
    # Replace out of vocab words with index for '<UNK>' tag
    vec_texts.append([vocab2idx.get(word,vocab2idx['<UNK>']) for word in text])
    vec_summaries.append([vocab2idx.get(word,vocab2idx['<UNK>']) for word in summary])

In [0]:
import random
random.seed(101)

texts_idx = [idx for idx in range(len(vec_texts))]
random.shuffle(texts_idx)

vec_texts = [vec_texts[idx] for idx in texts_idx]
vec_summaries = [vec_summaries[idx] for idx in texts_idx]

In [0]:
print(len(vec_texts))
print(len(vec_summaries))

15828
15828


In [0]:
# Use first 10000 data for testing, the next 10000 data for validation, and rest for training

test_summaries = vec_summaries[0:10000]
test_texts = vec_texts[0:10000]

val_summaries = vec_summaries[10000:20000]
val_texts = vec_texts[10000:20000]

train_summaries = vec_summaries[20000:]
train_texts = vec_texts[20000:]

In [0]:
def bucket_and_batch(texts,summaries,batch_size=32):
    
    # Sort summaries and texts according to the length of text
    # (So that texts with similar lengths tend to remain in the same batch and thus require less padding)
    
    text_lens = [len(text) for text in texts]
    sortedidx = np.flip(np.argsort(text_lens),axis=0)
    texts=[texts[idx] for idx in sortedidx]
    summaries=[summaries[idx] for idx in sortedidx]
    
    batches_text=[]
    batches_summary=[]
    batches_true_text_len = []
    batches_true_summary_len = []
    
    i=0
    while i < (len(texts)-batch_size):
        
        max_len = len(texts[i])
        
        batch_text=[]
        batch_summary=[]
        batch_true_text_len=[]
        batch_true_summary_len=[]
        
        for j in range(batch_size):
            
            padded_text = texts[i+j]
            padded_summary = summaries[i+j]
            
            batch_true_text_len.append(len(texts[i+j]))
            batch_true_summary_len.append(len(summaries[i+j])+1)
     
            while len(padded_text) < max_len:
                padded_text.append(vocab2idx['<PAD>'])

            padded_summary.append(vocab2idx['<EOS>']) #End of Sentence Marker
            while len(padded_summary) < summary_max_len+1:
                padded_summary.append(vocab2idx['<PAD>'])
            
        
            batch_text.append(padded_text)
            batch_summary.append(padded_summary)
        
        batches_text.append(batch_text)
        batches_summary.append(batch_summary)
        batches_true_text_len.append(batch_true_text_len)
        batches_true_summary_len.append(batch_true_summary_len)
        
        i+=batch_size
        
    return batches_text, batches_summary, batches_true_text_len, batches_true_summary_len

In [0]:
train_batches_text, train_batches_summary, train_batches_true_text_len, train_batches_true_summary_len \
= bucket_and_batch(train_texts, train_summaries)

val_batches_text, val_batches_summary, val_batches_true_text_len, val_batches_true_summary_len \
= bucket_and_batch(val_texts, val_summaries)

test_batches_text, test_batches_summary, test_batches_true_text_len, test_batches_true_summary_len \
= bucket_and_batch(test_texts, test_summaries)

In [0]:
import json

d = {}

d["vocab"] = vocab2idx
d["embd"] = embd.tolist()
d["train_batches_text"] = train_batches_text
d["test_batches_text"] = test_batches_text
d["val_batches_text"] = val_batches_text
d["train_batches_summary"] = train_batches_summary
d["test_batches_summary"] = test_batches_summary
d["val_batches_summary"] = val_batches_summary
d["train_batches_true_text_len"] = train_batches_true_text_len
d["val_batches_true_text_len"] = val_batches_true_text_len
d["test_batches_true_text_len"] = test_batches_true_text_len
d["train_batches_true_summary_len"] = train_batches_true_summary_len
d["val_batches_true_summary_len"] = val_batches_true_summary_len
d["test_batches_true_summary_len"] = test_batches_true_summary_len


with open('/content/drive/My Drive/Colab Notebooks/summ/release/processed.json', 'w') as outfile: 
    json.dump(d, outfile)